In [37]:
import nltk
import re
import collections
import random
import tensorflow as tf

In [5]:
with open('../../data/misc/the_last_question.txt', 'rt') as f:
    lines = [l.strip() for l in f]

In [21]:
sents = []
for l in lines:
    sents.extend(nltk.sent_tokenize(l))

In [24]:
counter = collections.Counter()
for s in sents:
    counter.update(s)

In [41]:
id_to_char = ['<NULL>', '<START>', '<END>'] + [c for c, _ in counter.most_common()]
char_to_id = { c: cid for cid, c in enumerate(id_to_char) }

In [42]:
max_len = max(len(s) for s in sents) + 1

In [69]:
random.shuffle(sents)

'It was only two inches cubed and nothing in itself, but it was connected through hyperspace with the great Galactic AC that served all mankind.'

In [45]:
def create_example(s):
    x = [0] * max_len
    y = [0] * max_len
    for i in range(len(s)):
        x[i] = char_to_id[s[i - 1]] if i > 0 else char_to_id['<START>']
        y[i] = char_to_id[s[i]]
    x[len(s)] = char_to_id[s[-1]]
    y[len(s)] = char_to_id['<END>']
    return { 'x': x, 'y': y, 'len': len(s) }

In [55]:
examples = [create_example(s) for s in sents]

In [61]:
random.shuffle(examples)

In [68]:
dev_ex = examples[:40]
train_ex = examples[40:]
len(train_ex), len(dev_ex)

(335, 40)

In [59]:
sess = tf.InteractiveSession()

In [ ]:
def rnn(input_data, size):
    # sizes
    batch_size, input_len, _ = tf.shape(input_data)

    # GRU
    gru = tf.contrib.cudnn_rnn.CudnnGRU(
        num_layers = 1
        num_units = size,
        input_size = input_data.shape[-1].value)
    
    # variables
    gru_params = tf.Variable(
        tf.random_uniform([gru.params_size()], -0.1, 0.1),
        name = 'gru_params'
        validate_shape = False)
    gru_input_h = tf.Variable(
        tf.zeros([1, batch_size, size]),
        name = 'gru_input_h')
    
    # tranpose to time-major
    input_data = tf.transpose(input_data, perm = [1, 0, 2])     # [input_len, batch_size, size]
    
    # tile input states
    input_h = tf.tile(gru_input_h, [input_len, 1, 1])

    # compute GRU
    out, _ = gru(input_data, input_h, gru_params)
    
    return out